In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import display

attestation = pd.read_csv("data/attestation.csv")
aes = pd.read_csv("data/tinyaes.csv")

In [5]:
class Boundary:
    """A boundary is defined by a start and end symbol along with a name."""
    def __init__(self, name, start, end):
        self.name = name
        self.start = start
        self.end = end

In [6]:
"""Given a set of boundaries (in no order) gather the number of cycles it took
to reach the end of each of the boundaries, by returning a pandas dataframe of
the boundaries, their labels, and the cycle cost. This should work in the case
of multiple enclaves being started, running a user/runtime, and then destroying.
"""

def get_indices(bset, data):
    """Return the indicies of all the found boundaries.
    """
    start_symbols = [b.start for b in bset]
    end_symbols = [b.end for b in bset]
    indicies = []
    for i, row in data.iterrows():
        if row['symbol'] in start_symbols or row['symbol'] in end_symbols:
            if i not in indicies:
                indicies.append(i)
                
    return indicies

def get_results(indicies, bset, data):
    """Given indicies of interest and the corresponding data
    return the cycle count of each phase.
    """
    start_symbols = [b.start for b in bset]
    end_symbols = [b.end for b in bset]
    
    # focus the data
    data = data.iloc[indicies]
    # clean up table
    data["end"] = data["symbol"].shift(-1)
    data["cost"] = -(data['cycle'] - data['cycle'].shift(-1))
    data = data.drop(['cycle'], axis=1)[:-1]
    # boundary matching
    labels = []
    for i, row in data.iterrows():
        start = row['symbol']
        end = row['end']
        for b in bset:
            if b.start == start and b.end == end:
                res = b.name
                break

        labels.append(res)
    
    # append labels in new column/series
    data["phase"] = pd.Series(labels).values
            
    return data

# Macro boundaries for the enclave
# We will also want to have smaller microboundaries for the enclave within these larger bucket.
# For these, we will not create separate boundaries.
# With the boundaries within boundaries, we can identify what's slowing down the enclave.
create_bound = Boundary("create_enclave", "keystone_create_enclave", "keystone_run_enclave")
run_bound = Boundary("run_enclave", "keystone_run_enclave", "rt_base")
runtime_bound = Boundary("enclave_runtimeuser1", "rt_base", "mcall_sm_exit_enclave")
attest_bound = Boundary("enclave_attestation", "mcall_sm_attest_enclave", "return_to_encl")
return_to_runtime_bound = Boundary("enclave_runtimeuser2", "return_to_encl", "mcall_sm_stop_enclave")
attest_edge_call_bound = Boundary("enclave_attestation_edge_call", "mcall_sm_stop_enclave", "return_to_encl")
run_to_exit_bound = Boundary("enclave_runtimeuser3", "return_to_encl", "mcall_sm_exit_enclave")
destroy_bound = Boundary("destroy_enclave", "destroy_enclave", "keystone_destroy_enclave_end")

enclave_bset = [create_bound, run_bound, runtime_bound, attest_bound, return_to_runtime_bound, attest_edge_call_bound,
                run_to_exit_bound, destroy_bound]

native_bset = [] # we just want user mode?

In [7]:
# Run Analysis
def data_pipeline(data):
    bset = enclave_bset
    indicies = get_indices(bset, data)
    results = get_results(indicies, bset, data)
    return results.groupby("phase").sum()

In [8]:
attres = data_pipeline(attestation)
attres

/usr/lib/python2.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/lib/python2.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,cost
phase,
create_enclave,49513319.0
enclave_attestation,1276616.0
enclave_attestation_edge_call,4908128.0
enclave_runtimeuser2,6715.0
enclave_runtimeuser3,2251.0
run_enclave,8681.0


In [14]:
aesres = data_pipeline(aes)
aesres

,cost
phase,
create_enclave,45165080.0
destroy_enclave,15320.0
run_enclave,540103.0


In [17]:
import plotly.plotly as py
import cufflinks as cf

# You'll need to set API key using:
# plotly.tools.set_credentials_file(username='DemoAccount', api_key='lr1c37zw81')
# Find your api_key here: https://plot.ly/settings/api
# Cufflinks binds plotly to pandas dataframes in IPython notebook.

cf.set_config_file(offline=False, world_readable=True, theme='ggplot')

attres.iplot(kind='bar', barmode='stack')

In [18]:
aesres.iplot(kind='bar', barmode='stack')